In [ ]:
# Baixar arquivos adicionais para o laboratório
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/nn_adv/class_01/Laborat%C3%B3rios/lab_utils_ml_adv_week_1.zip
      
!unzip -n -q lab_utils_ml_adv_week_1.zip

In [ ]:
# Testar se estamos no Google Colab
try:
  import google.colab
  IN_COLAB = True
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  IN_COLAB = False

# Rede Neural Simples com Numpy
Neste laboratório, criaremos uma pequena rede neural usando o Numpy. Será a mesma rede de "torrefação de café" que você implementou no Tensorflow.
   <center> <img  src="./images/C2_W1_CoffeeRoasting.png" width="400" />   <center/>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from lab_utils_common import dlc, sigmoid
from lab_coffee_utils import load_coffee_data, plt_roast, plt_prob, plt_layer, plt_network, plt_output_unit
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

## Conjunto de Dados
Este é o mesmo conjunto de dados do laboratório anterior.

In [ ]:
X,Y = load_coffee_data();
print(X.shape, Y.shape)

Vamos plotar os dados de torrefação de café abaixo. Os dois recursos são Temperatura em Celsius e Duração em minutos. A [Coffee Roasting at Home](https://www.merchantsofgreencoffee.com/how-to-roast-green-coffee-in-your-oven/) sugere que a duração deve ser mantida entre 12 e 15 minutos, enquanto a temperatura deve estar entre 175 e 260 graus Celsius. É claro que, à medida que a temperatura aumenta, a duração deve diminuir.

In [ ]:
plt_roast(X,Y)

### Normalizar dados
Para corresponder ao laboratório anterior, normalizaremos os dados. Consulte o laboratório anterior para obter mais detalhes

In [ ]:
print(f"Temperature Máxima, Min antes da normalização: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min antes da normalização: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)  # learns mean, variance
Xn = norm_l(X)
print(f"Temperature Max, Min após a normalização: {np.max(Xn[:,0]):0.2f}, {np.min(Xn[:,0]):0.2f}")
print(f"Duration    Max, Min após a normalização: {np.max(Xn[:,1]):0.2f}, {np.min(Xn[:,1]):0.2f}")

## Modelo Numpy - Propagação direta(_Forward Propagation_) no NumPy
<center> <img  src="./images/C2_W1_RoastingNetwork.PNG" width="200" />   <center/>  
Vamos criar a "Rede de torrefação de café" descrita na aula. Há duas camadas com ativações sigmoides.

Conforme descrito na aula, é possível criar sua própria camada densa usando o NumPy. Isso pode ser utilizado para criar uma rede neural de várias camadas.

<img src="images/C2_W1_dense3.png" width="600" height="450">

No primeiro laboratório você construiu um neurônio no NumPy e no Tensorflow e observou a semelhança entre eles. Uma camada simplesmente contém vários neurônios/unidades. Conforme descrito na aula, é possível utilizar um loop for para visitar cada unidade (`j`) na camada e executar o produto escalar dos pesos dessa unidade (`W[:,j]`) e somar o viés  da unidade (`b[j]`) para formar `z`. Uma função de ativação `g(z)` pode então ser aplicada a esse resultado. Vamos tentar isso abaixo para criar uma sub-rotina de "camada densa".

Primeiro, você definirá a função de ativação `g()`. Você usará a função `sigmoid()` que já está implementada para você no arquivo `lab_utils_common.py` fora deste notebook.

In [ ]:
# Definir a função de ativação
g = sigmoid

Em seguida, você definirá a função `my_dense()` que calcula as ativações de uma camada densa.

In [ ]:
def my_dense(a_in, W, b):
    """
    Calcula a camada densa
    Args:
      a_in (ndarray (n, )) : Dados, 1 exemplo
      W    (ndarray (n,j)) : Matriz de peso, n recursos por unidade, j unidades
      b    (ndarray (j, )) : vetor de bias, j unidades
    Returns
      a_out (ndarray (j,))  : j unidades|
    """
    units = W.shape[1]
    a_out = np.zeros(units)
    for j in range(units):               
        w = W[:,j]                                    
        z = np.dot(w, a_in) + b[j]         
        a_out[j] = g(z)               
    return(a_out)

*Observação: você também pode implementar a função acima para aceitar `g` como um parâmetro adicional (por exemplo, `my_dense(a_in, W, b, g)`). No entanto, neste notebook, você usará apenas um tipo de função de ativação (ou seja, sigmoide), portanto, não há problema em torná-la constante e defini-la fora da função. Foi o que você fez no código acima e isso simplifica as chamadas de função nas próximas células de código. Lembre-se de que passá-lo como um parâmetro também é uma implementação aceitável. Você verá isso na tarefa desta semana.*

A célula a seguir cria uma rede neural de duas camadas utilizando a sub-rotina `my_dense` acima.

In [ ]:
def my_sequential(x, W1, b1, W2, b2):
    a1 = my_dense(x,  W1, b1)
    a2 = my_dense(a1, W2, b2)
    return(a2)

Podemos copiar os pesos e os vieses treinados no laboratório anterior no Tensorflow.

In [ ]:
W1_tmp = np.array( [[-8.93,  0.29, 12.9 ], [-0.1,  -7.32, 10.81]] )
b1_tmp = np.array( [-9.82, -9.28,  0.96] )
W2_tmp = np.array( [[-31.18], [-27.59], [-32.56]] )
b2_tmp = np.array( [15.41] )

### Predições
<img align="left" src="./images/C2_W1_RoastingDecision.PNG"     style=" width:380px; padding: 10px 20px; " >

Quando você tiver um modelo treinado, poderá usá-lo para fazer previsões. Lembre-se de que o resultado do nosso modelo é uma probabilidade. Nesse caso, a probabilidade de uma boa torrefação. Para tomar uma decisão, é preciso aplicar a probabilidade a um limiar. Nesse caso, usaremos 0,5

Vamos começar escrevendo uma rotina semelhante ao `model.predict()` do Tensorflow. Isso pegará uma matriz $X$ com todos os $m$ exemplos nas linhas e fará uma previsão executando o modelo.

In [ ]:
def my_predict(X, W1, b1, W2, b2):
    m = X.shape[0]
    p = np.zeros((m,1))
    for i in range(m):
        p[i,0] = my_sequential(X[i], W1, b1, W2, b2)
    return(p)

Podemos testar essa rotina em dois exemplos:

In [ ]:
X_tst = np.array([
    [200,13.9],  # exemplo positivo
    [200,17]])   # exemplo negativo
X_tstn = norm_l(X_tst)  # lembre-se de normalizar
predictions = my_predict(X_tstn, W1_tmp, b1_tmp, W2_tmp, b2_tmp)

Para converter as probabilidades em uma decisão, aplicamos um limiar:

In [ ]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

Isso pode ser feito de forma mais sucinta:

In [ ]:
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

## Função de rede

Esse gráfico mostra a operação de toda a rede e é idêntico ao resultado do Tensorflow do laboratório anterior.
O gráfico à esquerda é a saída bruta da camada final representada pelo sombreamento azul. Ela é sobreposta aos dados de treinamento representados pelos Xs e Os.   
O gráfico da direita é a saída da rede após um limiar de decisão. Os Xs e Os aqui correspondem às decisões tomadas pela rede.

In [ ]:
netf= lambda x : my_predict(norm_l(x),W1_tmp, b1_tmp, W2_tmp, b2_tmp)
plt_network(X,Y,netf)

## Parabéns!
Você criou uma pequena rede neural no NumPy. 
Espero que este laboratório tenha revelado as funções bastante simples e familiares que compõem uma camada em uma rede neural.